In [1]:
import sys
sys.path.insert(0, '..')

import os
import requests


model_filepath = '../biosimulator_processes/model_files/BIOMD0000000061_url.xml'
print(os.path.exists(model_filepath))

tumor_control_model = 'BIOMD0000000749'
requests.get('https://www.ebi.ac.uk/biomodels/BIOMD0000000749', headers={'accept': 'json'})

True


<Response [200]>

In [2]:
from dataclasses import dataclass, asdict
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder, BuildPrompter
from biosimulator_processes.data_model import TimeCourseModel, TimeCourseModelChanges, ModelChanges, ModelChange

CobraProcess registered successfully.
CopasiProcess registered successfully.
SmoldynProcess registered successfully.
TelluriumProcess registered successfully.


#### Define a simple process composition configuration according to the CopasiProcess implementation (which is known by the type system/implementation as 'config_schema') with a SBML model filepath via the highest-level BioBuilder API:

#### Experiment 1:  Here we cross a boundary in the stack that is a biological simulation. We go from model configuration, to experiment. Thus, this tooling sits at that level: both experiment specification AND experiment execution. It's not just a way to specify an experiment, but it is also a way to run it, given the many knobs and buttons that you can use predefine and customize the way the actual model is being solved. We should make a bigger distinction between terms like "model".

In [3]:
# 1. define a model for the process composition. In this case, just one model to be re-used as configuration for the processes we create.

tc_model = TimeCourseModel(model_source=model_filepath)

In [6]:
  tc_model

TimeCourseModel(model_source='../biosimulator_processes/model_files/BIOMD0000000061_url.xml', model_id='BIOMD0000000061_url.xml_Model', model_name='BIOMD0000000061_url.xml_Model', model_language='sbml', model_changes=None, model_units=None)

In [5]:
tc_model.model_name

'BIOMD0000000061_url.xml_Model'